In [6]:
import torch
from einops import rearrange

In [7]:
torch_attention = torch.nn.MultiheadAttention(16, 4, 0, batch_first=True)

In [8]:
k, v, q = torch.rand(10, 3, 16), torch.rand(10, 3, 16), torch.rand(10, 5, 16)
o, _ = torch_attention(q, k, v)
print(o.shape)
print([(n, v.shape) for n, v in list(torch_attention.named_parameters())])

torch.Size([10, 5, 16])
[('in_proj_weight', torch.Size([48, 16])), ('in_proj_bias', torch.Size([48])), ('out_proj.weight', torch.Size([16, 16])), ('out_proj.bias', torch.Size([16]))]


In [12]:
class MHA(torch.nn.Module):
    def __init__(self, embed_dim, num_heads):
        super().__init__()
        self.in_proj_k = torch.nn.Linear(embed_dim, embed_dim)
        self.in_proj_v = torch.nn.Linear(embed_dim, embed_dim)
        self.in_proj_q = torch.nn.Linear(embed_dim, embed_dim)
        self.out_proj = torch.nn.Linear(embed_dim, embed_dim)
        self.num_heads = num_heads

    def forward(self, q, k, v, mask=None):
        p_q, p_k, p_v = self.in_proj_q(q), self.in_proj_k(k), self.in_proj_v(v)

        r_q = rearrange(p_q, 'b n (h d) -> b h n d', h=self.num_heads)
        r_k = rearrange(p_k, 'b n (h d) -> b h n d', h=self.num_heads)
        r_v = rearrange(p_v, 'b n (h d) -> b h n d', h=self.num_heads)

        scores = torch.einsum('b h m d, b h n d -> b h m n', r_q, r_k)
        attn = torch.nn.functional.softmax(scores, dim=-1)
        result = torch.einsum('b h m n, b h n d -> b h m d', attn, r_v)
        r_result = rearrange(result, 'b h m d -> b m (h d)')
        return self.out_proj(r_result)

our_attention = MHA(16, 4)
o_our = our_attention(q, k, v)
o_torch, _ = torch_attention(q, k, v)
print(o_our.shape, o_torch.shape)


torch.Size([10, 5, 16]) torch.Size([10, 5, 16])
